**Notebook final**

# 99 - modelo solución.ipynb



##Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#import local.lib.mlutils
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
%matplotlib inline

##Funciones propias

In [ ]:
#FUNCIONES:
def num_to_string(zcopy):
  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    1 : 'bajo',
    2 : 'medio-bajo',
    3 : 'medio-alto',
    4 : 'alto',
  }
  zcopy["RENDIMIENTO_GLOBAL"] = zcopy["RENDIMIENTO_GLOBAL"].replace(mapeo_valores)
  return zcopy



def string_to_num(zcopy):
  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
  }
  zcopy["RENDIMIENTO_GLOBAL"] = zcopy["RENDIMIENTO_GLOBAL"].replace(mapeo_valores)
  return zcopy



def mostrar_valores_unicos(dataframe, columna):
  valores_unicos = dataframe[columna].unique()
  print(f"Valores únicos en la columna {columna}:", valores_unicos.size)
  print(valores_unicos)



def reemplazar_si_no(dataframe, columna):
  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'Si': 1,
    'No': 0,
  }
  # Reemplazar los valores en la columna FAMI_ESTRATOVIVIENDA por los valores numéricos
  dataframe[columna] = dataframe[columna].replace(mapeo_valores)



def contar_nulos_si(dataframe, columna1, valor1, columna2):
  # Crear una máscara para identificar las filas donde ESTU_HORASSEMANATRABAJA es igual a cero
  mascara = (dataframe[columna1] == valor1)

  # Contar los valores nulos en la columna ESTU_PAGOMATRICULAPROPIO donde la máscara es True
  cantidad_nulos = dataframe.loc[mascara, columna2].isna().sum()
  print("Cantidad de valores nulos en ", columna2, " cuando ",columna1, " es ", valor1, ": ", cantidad_nulos)
  return None



def rellenar_nulos(dataframe, columna1, valor1, columna2, valor2):
  # Crear una máscara para identificar las filas donde ESTU_HORASSEMANATRABAJA es igual a cero
  mascara = (dataframe[columna1] == valor1)

  # Rellenar los valores nulos en la columna ESTU_PAGOMATRICULAPROPIO con cero en las filas donde la máscara es True
  dataframe.loc[mascara, columna2] = dataframe.loc[mascara, columna2].fillna(valor2)

  return None



def rellenar_nulos2(dataframe, columna1, valor1, columna2, valor2, columna3, valor3):

  # Crear una máscara para identificar las filas donde las dos columnas tienen los valores específicos
  mascara = (dataframe[columna1] == valor1) & (dataframe[columna2] == valor2)

  # Rellenar los valores nulos en la columna3 con valor3 en las filas donde la máscara es True
  dataframe.loc[mascara, columna3] = dataframe.loc[mascara, columna3].fillna(valor3)

  return None



def rellenar_nulos3(dataframe, columna1, valor1, columna2, valor2, columna3, valor3, columna4, valor4):

  # Crear una máscara para identificar las filas donde las tres columnas tienen los valores específicos
  mascara = (dataframe[columna1] == valor1) & (dataframe[columna2] == valor2) & (dataframe[columna3] == valor3)

  # Rellenar los valores nulos en la columna4 con valor4 en las filas donde la máscara es True
  dataframe.loc[mascara, columna4] = dataframe.loc[mascara, columna4].fillna(valor4)

  return None



def mostrar_valores_unicos_y_frecuencia(dataframe, columna, valores_a_mostrar=50):

  conteo_valores = dataframe[columna].value_counts(dropna=False)

  print("Conteo de valores en la columna ", columna, ": "  , conteo_valores.size)
  print(conteo_valores)
  print()
  print()
  conteo_valores.head(valores_a_mostrar)

  return None

#TOMADAS DEL NOTEBOOK ELABORADO POR EL PROFESOR RAUL
def to_onehot(x):
  values = np.unique(x)
  r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
  return np.eye(len(values))[r].astype(int)

def replace_columns_with_onehot(d, col):
  k = to_onehot(d[col].values)
  r = pd.DataFrame(k, columns=["%s_%d"%(col, i) for i in range(k.shape[1])], index=d.index).join(d)
  del(r[col])
  return r

In [ ]:
#graficas
import seaborn as sns
import matplotlib.pyplot as plt

def grafico_barras(df, ejex, ejey, inicio=0, elementos=0, tamaño_x=12, tamaño_y=6):

  # Crear un gráfico de barras
  #plt.figure(figsize=(tamaño_x, tamaño_y))
  num_categorias = len(df[ejex].value_counts())
  tamaño_x = min(tamaño_x, max(tamaño_y, num_categorias * 0.75))  # Tamaño mínimo de 6, máximo de 12
  tamaño_y = tamaño_x * 0.75

  # Crear un gráfico de barras
  plt.figure(figsize=(tamaño_x, tamaño_y))

  if elementos == 0:
    sns.countplot(data=df, x=ejex, hue=ejey, palette="Set2", order=df[ejex].value_counts().index)
  else:
    sns.countplot(data=df, x=ejex, hue=ejey, palette="Set2", order=df[ejex].value_counts().index[inicio:elementos])

  # Añadir etiquetas y título
  plt.xlabel(ejex)
  plt.ylabel(ejey)
  plt.title("Distribución del " + ejey + " por " + ejex)

  # Rotar las etiquetas del eje x para una mejor visualización
  plt.xticks(rotation=90)

  # Ajustar la leyenda
  plt.legend(title=ejey, loc="upper right")

  # Mostrar el gráfico
  plt.tight_layout()
  plt.show()

  return None

##Carga desde Kaggle

In [ ]:
#recordar ingresar el token.json
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip

 33% 7.00M/21.2M [00:00<00:00, 72.1MB/s]
100% 21.2M/21.2M [00:00<00:00, 143MB/s] 
Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
z = pd.read_csv('train.csv')
dts = pd.read_csv("test.csv")
dtr = z.copy()

dtr.shape, dts.shape

((692500, 17), (296786, 16))

##Función limpieza (preprocesado)

In [ ]:
def limpieza(dframe, train=True):
  zcopy = dframe.copy()
#----------------------------------BORRAR COLUMNAS----------------------------------
  #BORRAR COLUMNA ESTU_PRIVADO_LIBERTAD: porque todos los datos son iguales
  zcopy.drop(columns=["ESTU_PRIVADO_LIBERTAD"], inplace=True)
  #BORRAR COLUMNA FAMI_TIENEINTERNET.1: porque hay 2 columnas iguales
  zcopy.drop(columns=["FAMI_TIENEINTERNET.1"], inplace=True)
  #BORRAR COLUMNA ID: Identificador único de cada entrada. No aporta información predictiva.
  zcopy.drop(columns=["ID"], inplace=True)


#----------------------------------MAPEAR COLUMNAS----------------------------------

  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'Entre 5.5 millones y menos de 7 millones':6,
    'Entre 2.5 millones y menos de 4 millones':4,
    'Entre 4 millones y menos de 5.5 millones':5,
    'Más de 7 millones':7,
    'Entre 1 millón y menos de 2.5 millones':3,
    'Entre 500 mil y menos de 1 millón':2,
    'Menos de 500 mil':1,
    'No pagó matrícula':0,
  }
  zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"] = zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'No sabe' :0,
    'Ninguno':0,
    'No Aplica':0,
    'Primaria incompleta':0,
    'Primaria completa':1,
    'Secundaria (Bachillerato) incompleta' :1,
    'Secundaria (Bachillerato) completa':2,
    'Técnica o tecnológica incompleta':2,
    'Educación profesional incompleta':2,
    'Técnica o tecnológica completa':3,
    'Educación profesional completa' :4,
    'Postgrado':5
  }
  zcopy["FAMI_EDUCACIONPADRE"] = zcopy["FAMI_EDUCACIONPADRE"].replace(mapeo_valores)
  zcopy["FAMI_EDUCACIONMADRE"] = zcopy["FAMI_EDUCACIONMADRE"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4,
  }
  zcopy["ESTU_HORASSEMANATRABAJA"] = zcopy["ESTU_HORASSEMANATRABAJA"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0, #PENDIENTE PA REVISAR                                                         PONERLO COMO ESTRATO 1?!!!!!
  }
  zcopy["FAMI_ESTRATOVIVIENDA"] = zcopy["FAMI_ESTRATOVIVIENDA"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para cada categoría
  columnas_si_no = ["FAMI_TIENEINTERNET", "FAMI_TIENELAVADORA", "FAMI_TIENEAUTOMOVIL", "ESTU_PAGOMATRICULAPROPIO", "FAMI_TIENECOMPUTADOR"]
  for columna in columnas_si_no:
    reemplazar_si_no(zcopy, columna)

#----------------------------------RELLENAR OBVIEDADES----------------------------------
  estratos = [4,5,6]
  for i in estratos:
    rellenar_nulos(zcopy, "FAMI_ESTRATOVIVIENDA", i, "FAMI_TIENEINTERNET", 1)
    rellenar_nulos(zcopy, "FAMI_ESTRATOVIVIENDA", i, "FAMI_TIENECOMPUTADOR", 1)
    rellenar_nulos(zcopy, "FAMI_ESTRATOVIVIENDA", i, "FAMI_TIENELAVADORA", 1)
    if i!=4:
      rellenar_nulos(zcopy, "FAMI_ESTRATOVIVIENDA", i, "FAMI_TIENEAUTOMOVIL", 1)


  rellenar_nulos(zcopy, "FAMI_ESTRATOVIVIENDA", 1, "FAMI_TIENEAUTOMOVIL", 0)

  rellenar_nulos(zcopy, "FAMI_TIENEAUTOMOVIL", 1, "FAMI_TIENELAVADORA", 1)
  rellenar_nulos(zcopy, "FAMI_TIENEAUTOMOVIL", 1, "FAMI_TIENEINTERNET", 1)
  rellenar_nulos(zcopy, "FAMI_TIENEAUTOMOVIL", 1, "FAMI_TIENECOMPUTADOR", 1)

  rellenar_nulos(zcopy, "FAMI_TIENECOMPUTADOR", 1, "FAMI_TIENEINTERNET", 1)

  rellenar_nulos(zcopy, "ESTU_HORASSEMANATRABAJA", 0, "ESTU_PAGOMATRICULAPROPIO", 0)

#----------------------------------ELIMINAR FILAS CON VALORES NAN----------------------------------
  if train:
    umbral = 8
    zcopy = zcopy[zcopy.isnull().sum(axis=1) < umbral]

  zcopy = zcopy.fillna(0)

#----------------------------------AGRUPAR CARRERAS POR FACULTADES----------------------------------
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('INGENIERIA|INGENIER¿A|INGENIERÍA|acui|ing|compu', case=False), 'ESTU_PRGM_ACADEMICO'] = 'ingenieria'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('licenciatura|peda', case=False), 'ESTU_PRGM_ACADEMICO'] = 'educacion'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('econom|contadur|empresas|contabilidad|internaci|finan|comerci|negocios|mark|empr|merc', case=False), 'ESTU_PRGM_ACADEMICO'] = 'economia y negocios'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('derecho|política|politica|crimin|investi|gob|pol|juris|desa', case=False), 'ESTU_PRGM_ACADEMICO'] = 'derecho'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('social|psicolog|sicolog|psi|periodismo|antropolog|fami|sociol|geog', case=False), 'ESTU_PRGM_ACADEMICO'] = 'ciencias sociales'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('veteri|zoo|agro', case=False), 'ESTU_PRGM_ACADEMICO'] = 'veterinaria'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('medic|salud|enfer|odonto|fisio|nutri|quiru|terap|fonoaud|bacte|geron|optom', case=False), 'ESTU_PRGM_ACADEMICO'] = 'salud'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('dise|art|mus|audi|teat|publicidad|produ|cine|dram|danz|maestro|foto|publ|mús|anim|band|mueb', case=False), 'ESTU_PRGM_ACADEMICO'] = 'artes'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('admi|hote|turi', case=False), 'ESTU_PRGM_ACADEMICO'] = 'administracion'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('gastro|culin', case=False), 'ESTU_PRGM_ACADEMICO'] = 'gastronomia'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('farm', case=False), 'ESTU_PRGM_ACADEMICO'] = 'farmacia'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('arqui|const|urba', case=False), 'ESTU_PRGM_ACADEMICO'] = 'arquitectura'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('depor|recre', case=False), 'ESTU_PRGM_ACADEMICO'] = 'deporte'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('filo|hist|lite|ling|arque|cult|bibli|arch|leng|teo', case=False), 'ESTU_PRGM_ACADEMICO'] = 'humanidades'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('quim|fis|bio|geol|mat|micr|eco|ASTRONOMIA|quí|fí|estad|geo|ambi', case=False), 'ESTU_PRGM_ACADEMICO'] = 'exactas y naturales'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('milit|mari|nav', case=False), 'ESTU_PRGM_ACADEMICO'] = 'fuerzas armadas'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('comu|narra|sis|info', case=False), 'ESTU_PRGM_ACADEMICO'] = 'comunicacion'

#----------------------------------AGRUPAR DEPARTAMENTOS POR REGIONES----------------------------------
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('antio|boya|cald|cundi|huila|bogo|santan|quind|risar|toli', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'andina'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('amazo|caque|guavia|putu|vau', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'amazonica'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('valle|choc|nari|cauc', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'pacifico'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('arau|casan|meta', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'orinoquia'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('atlantico|bolivar|cesar|cordoba|guajira|magdalena|sucre', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'caribe'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('andr', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'insular'

#----------------------------------LIMPIEZA ONE HOT----------------------------------
  zcopy = replace_columns_with_onehot(zcopy, "ESTU_PRGM_ACADEMICO")
  zcopy = replace_columns_with_onehot(zcopy, "ESTU_PRGM_DEPARTAMENTO")

  return zcopy

##df antes de la limpieza

In [ ]:
z.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


##df después de la limpieza

In [ ]:
zcopy = limpieza(dtr)

In [ ]:
Xts = limpieza(dts, train= False)

In [ ]:
zcopy = string_to_num(zcopy)
zcopy.head()

,ESTU_PRGM_DEPARTAMENTO_0,ESTU_PRGM_DEPARTAMENTO_1,ESTU_PRGM_DEPARTAMENTO_2,ESTU_PRGM_DEPARTAMENTO_3,ESTU_PRGM_DEPARTAMENTO_4,ESTU_PRGM_DEPARTAMENTO_5,ESTU_PRGM_ACADEMICO_0,ESTU_PRGM_ACADEMICO_1,ESTU_PRGM_ACADEMICO_2,ESTU_PRGM_ACADEMICO_3,...,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,0,1,0,0,0,0,0,0,0,0,...,1.0,3.0,1.0,2.0,1.0,1.0,0.0,1.0,5.0,3
1,0,0,1,0,0,0,0,0,0,0,...,0.0,3.0,0.0,3.0,1.0,0.0,0.0,1.0,2.0,1
2,0,1,0,0,0,0,0,0,0,0,...,4.0,3.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,1
3,0,1,0,0,0,0,0,0,0,0,...,0.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,4
4,0,1,0,0,0,0,0,0,0,1,...,3.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2


In [ ]:
Xts.head()

,ESTU_PRGM_DEPARTAMENTO_0,ESTU_PRGM_DEPARTAMENTO_1,ESTU_PRGM_DEPARTAMENTO_2,ESTU_PRGM_DEPARTAMENTO_3,ESTU_PRGM_DEPARTAMENTO_4,ESTU_PRGM_DEPARTAMENTO_5,ESTU_PRGM_ACADEMICO_0,ESTU_PRGM_ACADEMICO_1,ESTU_PRGM_ACADEMICO_2,ESTU_PRGM_ACADEMICO_3,...,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
0,0,0,1,0,0,0,0,0,0,1,...,1.0,1.0,3.0,1.0,3.0,1.0,0.0,1.0,1.0,1.0
1,0,1,0,0,0,0,0,0,0,0,...,4.0,3.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,3.0
2,0,1,0,0,0,0,0,0,0,0,...,3.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0
3,0,0,1,0,0,0,0,0,0,0,...,3.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,0,1,0,0,0,0,0,0,0,...,4.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,2.0


In [ ]:
print("Antes y después de la limpieza:")
print("dtrain", dtr.shape, "dtest", dts.shape)
# print("Después de la limpieza:")
print("dtrain", zcopy.shape, "dtest", Xts.shape)

Antes y después de la limpieza:
dtrain (692500, 17) dtest (296786, 16)
dtrain (671681, 33) dtest (296786, 32)


##Split de los datos (train -> train/validation)

In [ ]:
submission = dts["ID"].values

ytr = zcopy["RENDIMIENTO_GLOBAL"].values
zcopy.drop(columns=["RENDIMIENTO_GLOBAL"], inplace=True)
Xtr = zcopy

In [ ]:
print("Xtrain       ytrain")
print (Xtr.shape, ytr.shape)
print("Xtest        submission")
print (Xts.shape, submission.shape)

Xtrain       ytrain
(671681, 32) (671681,)
Xtest        submission
(296786, 32) (296786,)


In [ ]:
# TRAIN 80% ---- TEST 20%
X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.2, random_state=42)
print("X_train       X_test       y_train   y_test")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train       X_test       y_train   y_test
(537344, 32) (134337, 32) (537344,) (134337,)


##Elección modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
#from sklearn.pipeline import Pipeline


###Catboost

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.2 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=10, random_seed=42, verbose=100)
cb.fit(X_train, y_train)

# Calcular y mostrar el puntaje en los datos de entrenamiento
train_score_cb = cb.score(X_train, y_train)
print("Score en datos de TRAIN:", train_score_cb)

preds_cb = cb.predict(X_test)

# Calcular y mostrar el puntaje en los datos de prueba
test_score_cb = accuracy_score(y_test, preds_cb)
print("Score en datos de TEST:", test_score_cb)

0:	learn: 1.3648633	total: 795ms	remaining: 13m 14s
100:	learn: 1.2176673	total: 1m 17s	remaining: 11m 32s
200:	learn: 1.1992867	total: 2m 34s	remaining: 10m 16s
300:	learn: 1.1831270	total: 3m 55s	remaining: 9m 6s
400:	learn: 1.1692198	total: 5m 12s	remaining: 7m 46s
500:	learn: 1.1555834	total: 6m 29s	remaining: 6m 27s
600:	learn: 1.1435053	total: 7m 52s	remaining: 5m 13s
700:	learn: 1.1309946	total: 9m 9s	remaining: 3m 54s
800:	learn: 1.1197928	total: 10m 25s	remaining: 2m 35s
900:	learn: 1.1084321	total: 11m 46s	remaining: 1m 17s
999:	learn: 1.0980458	total: 13m 2s	remaining: 0us
Score en datos de TRAIN: 0.5277903912577417
Score en datos de TEST: 0.40448275605380496


In [ ]:
#el mejor kaggle 5
from catboost import CatBoostClassifier

cb1 = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=5, random_seed=42, verbose=100)
cb1.fit(X_train, y_train)

# Calcular y mostrar el puntaje en los datos de entrenamiento
train_score_cb1 = cb1.score(X_train, y_train)
print("Score en datos de TRAIN:", train_score_cb1)

preds_cb1 = cb1.predict(X_test)

# Calcular y mostrar el puntaje en los datos de prueba
test_score_cb1 = accuracy_score(y_test, preds_cb1)
print("Score en datos de TEST:", test_score_cb1)

0:	learn: 1.3693388	total: 318ms	remaining: 2m 38s
100:	learn: 1.2390614	total: 27.5s	remaining: 1m 48s
200:	learn: 1.2320587	total: 54.5s	remaining: 1m 21s
300:	learn: 1.2288930	total: 1m 20s	remaining: 53.3s
400:	learn: 1.2267524	total: 1m 49s	remaining: 27.1s
499:	learn: 1.2251435	total: 2m 16s	remaining: 0us
Score en datos de TRAIN: 0.42275525547879944
Score en datos de TEST: 0.4159911267930652


In [ ]:
#6 el mejor
from catboost import CatBoostClassifier

cb2 = CatBoostClassifier(iterations=400, learning_rate=0.1, depth=6, random_seed=42, verbose=100)
cb2.fit(Xtr, ytr)

# Calcular y mostrar el puntaje en los datos de entrenamiento
train_score_cb2 = cb2.score(Xtr, ytr)
print("Score en datos de TRAIN:", train_score_cb2)

preds_cb2 = cb2.predict(X_test)

# Calcular y mostrar el puntaje en los datos de prueba
test_score_cb2 = accuracy_score(y_test, preds_cb2)
print("Score en datos de TEST:", test_score_cb2)

0:	learn: 1.3682695	total: 581ms	remaining: 3m 51s
100:	learn: 1.2359301	total: 38.2s	remaining: 1m 53s
200:	learn: 1.2293222	total: 1m 12s	remaining: 1m 12s
300:	learn: 1.2259173	total: 1m 49s	remaining: 35.9s
399:	learn: 1.2234212	total: 2m 26s	remaining: 0us
Score en datos de TRAIN: 0.4235239645010057
Score en datos de TEST: 0.4223110535444442


**fill con el mejor modelo**




In [ ]:
preds = cb2.predict(Xts)
preds = preds.ravel()


##Generación del Kaggle submission

In [ ]:
submission = pd.DataFrame([dts["ID"], pd.Series(preds, name="RENDIMIENTO_GLOBAL")]).T

submission = num_to_string(submission)
submission['ID'] = submission['ID'].astype(int)

submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
submission.to_csv("saberPRO_kaggle.csv", index=False)

In [ ]:
#!head saberPRO_kaggle.csv